# WORKING

---

## Setup

In [1]:
import numpy as np
import pandas as pd

In [2]:
INPUT_FILE = 'input.txt'

In [3]:
with open(INPUT_FILE, 'r') as f:
    lines = [line.strip('\n') for line in f.readlines()]
    
# lines

---

### Create Tree Grid

In [4]:
def create_tree_grid(lines: list[str]) -> list[list[int]]:
    """Create an array from a list of numeric strings."""
    rows = [[int(num) for num
             in list(line)]
            for line in lines]
    # Convert to numpy array so object can be
    # transposed to check columns later on
    rows = np.array(rows)
    
    return rows

---

### Create (empty) Checked Grid

In [5]:
def create_checked_grid(tree_grid):
    """Create a grid with the same shape as the tree_grid
    array, filled with 0s to represent tree locations in the grid.
    Visible trees will be denoted on this grid with a 1."""
    num_of_rows = tree_grid.shape[0]
    num_of_columns = tree_grid.shape[1]
    
    checked_grid = np.zeros((num_of_rows, num_of_columns), dtype=np.int16)   

    return checked_grid

---

#### Example Tree Grid

In [6]:
# Example input, same format as "lines"

example_tree_grid = [
    '30373',
    '25512',
    '65332',
    '33549',
    '35390'
]

In [7]:
tree_grid = create_tree_grid(example_tree_grid)
# tree_grid = create_tree_grid(lines)
tree_grid 

array([[3, 0, 3, 7, 3],
       [2, 5, 5, 1, 2],
       [6, 5, 3, 3, 2],
       [3, 3, 5, 4, 9],
       [3, 5, 3, 9, 0]])

---

#### Empty Checked Grid

In [8]:
empty_checked_grid = create_checked_grid(tree_grid)
empty_checked_grid

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int16)

---

## Check Across (Left-to-Right and Right-to-Left) Rows for Visible Trees

In [39]:
def check_row_left_to_right(tree_grid):
        # Save row, column point for each visible tree to this list
        all_visible = []
        
        # If the tree is in the first or last row or column (edge tree) add its 
        # point to the list of visible trees
        for i, row in enumerate(tree_grid):    
            for j, tree in enumerate(row):
                if i == 0 or j == 0 or i == row_last_index or j == row_last_index:
                    all_visible.append([i, j])

        # Enumerate through each row in the tree grid
        for i, row in enumerate(tree_grid):
            # Find the tallest tree for that row
            tallest_tree_height = max(row)

            # Enumerate through the trees in each row
            for j, tree in enumerate(row):
                # If the tree is the first tree in the row
                if j == 0:
                    # The first tree we check is the current tallest tree of the checked trees
                    current_tree = row[j]
                    all_visible.append([i, j])
                    # But if the current tree is the first tallest tree, then stop there.
                    if current_tree == tallest_tree_height:
                        break
                    
                # If the tree is not the first in the row and the tree equals the 
                # tallest tree height 
                elif tree == tallest_tree_height:
                    # Add that tree's point to the list of visible trees
                    all_visible.append([i, j])
                    # And then stop checking the trees in that row, skip to the next row
                    break
                
                # Otherwise, if the tree is not the first tree in the row and it does not
                # equal the tallest tree
                elif tree != tallest_tree_height:
                    # If the tree's height is taller than the current tree's height
                    if tree > current_tree:
                        # Add that tree's points to the list of visible trees
                        all_visible.append([i, j])
                        # Then, set the current tree to that tree, so that any tree after
                        # it must be higher than it to be visible
                        current_tree = tree

                    # If the tree is not the first tallest tree in the row and it is not taller
                    # than the current tallest tree, it is not visible, so continue checking the
                    # trees in the current row until the tallest tree is found.
                    elif tree <= current_tree:
                        continue

        # Returns a list of lists, where each sublist is a coordinate for a tree in the grid,
        # [row, column] to be represented in the checked_grid as a 1.                    
        return all_visible

---

## Add Coordinates to Checked Grid

In [48]:
def add_coordinates_to_checked_grid(all_coordinates):
    
    checked_grid = create_checked_grid(tree_grid)
 
    for coordinate in all_coordinates:
        row, column = coordinate
        checked_grid[row][column] = 1
        
    return checked_grid

---

## Get Coordinates

---

### Rows 

---

#### Left-to-Right

In [49]:
left_to_right_coordinates = check_row(tree_grid, is_reversed=False)
left_to_right_coordinates

[[0, 0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [1, 0],
 [1, 4],
 [2, 0],
 [2, 4],
 [3, 0],
 [3, 4],
 [4, 0],
 [4, 1],
 [4, 2],
 [4, 3],
 [4, 4],
 [0, 0],
 [0, 3],
 [1, 0],
 [1, 1],
 [2, 0],
 [3, 0],
 [3, 2],
 [3, 4],
 [4, 0],
 [4, 1],
 [4, 3]]

In [51]:
left_to_right_grid = add_coordinates_to_checked_grid(left_to_right_coordinates)
left_to_right_grid

array([[1, 1, 1, 1, 1],
       [1, 1, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1]], dtype=int16)

---

#### Right-to-Left

In [52]:

right_to_left_coordinates = check_row(tree_grid, is_reversed=True)
right_to_left_coordinates

[[0, 0],
 [0, 1],
 [1, 0],
 [1, 2],
 [2, 0],
 [2, 1],
 [2, 3],
 [2, 4],
 [3, 0],
 [4, 0],
 [4, 1]]

In [47]:
right_to_left_grid = add_coordinates_to_checked_grid(right_to_left_coordinates, empty_checked_grid)
right_to_left_grid

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1]], dtype=int16)

---

### Columns

---

## Transpose Grid to check Columns

In [18]:
tree_grid_transposed = np.transpose(tree_grid)
tree_grid_transposed

array([[3, 2, 6, 3, 3],
       [0, 5, 5, 3, 5],
       [3, 5, 3, 5, 3],
       [7, 1, 3, 4, 9],
       [3, 2, 2, 9, 0]])

In [34]:
up_to_down_coordinates = check_row(tree_grid, is_reversed=False)
down_to_up_coordinates = check_row(tree_grid, is_reversed=True)

In [20]:
all_coordinates = get_all_coordinates(up_to_down_coordinates, down_to_up_coordinates)

In [25]:
add_coordinates_to_checked_grid(all_coordinates, half_checked_grid)

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1]], dtype=int16)

In [17]:
true_checked_grid = [
    [1, 1, 1, 1, 1],
    [1, 1, 1, 0, 1],
    [1, 1, 0, 1, 1],
    [1, 0, 1, 0, 1],
    [1, 1, 1, 1, 1]    
]

checked_grid == true_checked_grid

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

---

## WORKING

In [ ]:
def add_edge_trees_to_checked_grid(tree_grid):

    last_row_index = len(tree_grid) - 1

    # Check every row in the tree grid, and keep track of the index of the current row
    for row_number, row in enumerate(tree_grid):
        # Check every tree in each row, and keep track of the index of the current tree    
        for tree_number, tree in enumerate(row):
            # Add a 1 to the tree's location in the checked grid if the tree is on the edge
            # of the grid
            if (
                (row_number == 0) or (row_number == last_row_index) or 
                (tree_number == 0) or (tree_number == len(row) - 1)
            ):
                checked_grid[row_number][tree_number] = 1

            # Otherwise, keep track of each inner tree's neighbors.

    return checked_grid    

In [31]:
a = [[0, 0, 0],
     [1, 1, 0], 
     [0, 1, 0]]

a = np.array(a, dtype=bool)
a

b = [[0, 1, 1],
     [0, 0, 1],
     [1, 0, 1]]

b = np.array(b, dtype=bool)

(a|b)

array([[False, False, False],
       [False, False, False],
       [False, False, False]])

In [53]:
def check_row_left_to_right(tree_grid):
    """Create a grid of trees visible for each row when viewed from left to right."""
    
    # Initialize empty grid to add visible trees (1s) to
    checked_grid_left_to_right = create_empty_checked_grid()
    
    # For each row in the tree grid
    for i, row in enumerate(tree_grid):
        # Find the tallest tree for that row
        tallest_tree_height = max(row)
        
        # Iterate through each tree in the row
        for j, tree in enumerate(row):
            # For the first tree in the row, set it as the current tallest tree
            if j == 0:
                current_tree = row[j]
                checked_grid_left_to_right[i][j] = 1
                # If the first tree is the tallest tree, skip to checking the next row
                if current_tree == tallest_tree_height:
                    break
            
            # If the tree is not the first in the row, and the tree equals to the
            # tallest tree height
            elif tree == tallest_tree_height:
                # Add a visible tree marker (1) to the checked_trees grid at the tree's
                # coordinates
                checked_grid_left_to_right[i][j] = 1
                # Then skip to checking the next row
                break
            
            # If the tree is not the first in the row, and it is not the tallest tree
            elif tree != tallest_tree_height:
                # If the tree's height is taller than the current tallest tree
                if tree > current_tree:
                    # Add a visible tree marker (1) to the checked_trees grid
                    checked_grid_left_to_right[i][j] = 1
                    # Then, change the current_tree to that tree, so that each following tree
                    # must be taller than it to be considered visible.
                    current_tree = tree
                    
                # The tree is not visible if it is not taller than the current tallest tree,
                # so continue checking the rest of the trees in the current row until the
                # tallest tree is found.
                elif tree <= current_tree:
                    continue
                
    return checked_grid_left_to_right